In [1]:
# imports
import cv2
import natsort
import numpy as np
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern

In [2]:
def calculate_lbp(image, quadrants=(1,1), neighbours=8, radius=2):
  # Tamanho dos quadrantes
  width, height = image.shape[0], image.shape[1]
  part_width = width // quadrants[0]
  part_height = height // quadrants[1]
  
  # Calculate
  result = list()
  for i in range(quadrants[0]):
    for j in range(quadrants[1]):
      #coordenadas da parte da imagem
      left = i * part_width
      upper = j * part_height
      right = left + part_width
      lower = upper + part_height

      # Retorna o quadrante da imagem
      quadrant = image[left:right, upper:lower]

      # Calculate Features
      lbp_image = local_binary_pattern(quadrant, neighbours, radius, method="nri_uniform")
      n_bins = int(lbp_image.max() + 1)
      hist, _ = np.histogram(lbp_image, density=True, bins=n_bins, range=(0, n_bins))

      # Append to result
      result.extend(hist)
          
  return result
#end lbp

def calculate_glcm(image, quadrants=(1,1)):
  # Tamanho dos quadrantes
  width, height = image.shape[0], image.shape[1]
  part_width = width // quadrants[0]
  part_height = height // quadrants[1]
  
  # Calculate
  result = list()
  for i in range(quadrants[0]):
    for j in range(quadrants[1]):
      #coordenadas da parte da imagem
      left = i * part_width
      upper = j * part_height
      right = left + part_width
      lower = upper + part_height

      # Retorna o quadrante da imagem
      quadrant = image[left:right, upper:lower]

      # Calcula glcm matrix
      glcm = graycomatrix(quadrant, [1], [0], symmetric=True, normed=True)

      # Calculate Features
      props = [
        graycoprops(glcm, prop='contrast')[0][0],
        graycoprops(glcm, prop='dissimilarity')[0][0],
        graycoprops(glcm, prop='homogeneity')[0][0],
        graycoprops(glcm, prop='ASM')[0][0],
        graycoprops(glcm, prop='energy')[0][0],
        graycoprops(glcm, prop='correlation')[0][0],
      ]

      # Append to result
      result.extend(props)
          
  return result

In [3]:
def get_files(caminho_p):
  file_dict = {}
  for pasta in Path(caminho_p).iterdir():
    if pasta.is_dir():
      file_dict[pasta.name] = []
            
      for file in Path(pasta).iterdir():
        if file.is_file() and (file.name.endswith(".png") or file.name.endswith(".jpg") or file.name.endswith(".bmp")):
          file_dict[pasta.name].append(file)
  return file_dict
#end getfilos


def createXy(database, quadrants=(1,1), lbp_props={'neighbours': 8, 'radius': 2}):
  X_glcm, X_lbp, y = list(), list(), list()

  for classe in database:   
    for item in natsort.natsorted(database[classe]):
      image = cv2.imread(str(item), cv2.IMREAD_GRAYSCALE)

      X_glcm.append(calculate_glcm(image, quadrants))
      X_lbp.append(calculate_lbp(image, quadrants, lbp_props['neighbours'], lbp_props['radius']))
      y.append(classe)

  return np.array(X_glcm), np.array(X_lbp), np.array(y)
#end createxy


def split_normalizado(X, y, test_size=0.4, shuffle=True):
  X = StandardScaler().fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=shuffle, stratify=None)
  return X_train, X_test, y_train, y_test
#end treinotest


def split_manual(X, y, train_size=18):
  #normalize
  X = StandardScaler().fit_transform(X)

  # separar em classes
  classes = dict()
  for i, classe in enumerate(y):
    if not classes.get(classe): classes[classe] = list()
    classes[classe].append(X[i])

  #split
  X_train, X_test, y_train, y_test = list(), list(), list(), list()

  for classe in classes.keys():
    for i, instancia in enumerate(classes[classe]):
      if i <= train_size:
        X_train.append(instancia)
        y_train.append(classe)
      else:
        X_test.append(instancia)
        y_test.append(classe)

  return X_train, X_test, y_train, y_test
#end treinotest

In [4]:
def knn(X_train, y_train, X_test, props={
  'n_neighbors': 5,
  'metric': 'euclidean',
}):
  knn = KNeighborsClassifier(n_neighbors=props['n_neighbors'], metric=props['metric']).fit(X_train, y_train)
  return knn.predict(X_test)
#end knn

def svm(X_train, y_train, X_test, props={
  'C': 1.0,
  'kernel': 'rbf',
  'gamma': 'scale',
  'coef0': 0
}):
  clf = SVC(C=props['C'], kernel=props['kernel'], gamma=props['gamma'], coef0=props['coef0'])
  clf.fit(X_train, y_train)
  return clf.predict(X_test)

In [5]:
# database = get_files('../datasets/Base_BFL_256/')
# X_glcm, X_lbp, y = createXy(database, quadrants=(3, 3))

# print(X_glcm.shape, X_lbp.shape, y.shape)

# # # ========================= GLCM
# # X_glcm_train, X_glcm_test, y_glcm_train, y_glcm_test = split_normalizado(X_glcm, y, test_size=0.3)
# X_glcm_train, X_glcm_test, y_glcm_train, y_glcm_test = split_manual(X_glcm, y, train_size=18)

# y_glcm_pred = knn(X_glcm_train, y_glcm_train, X_glcm_test, props=knnProps)
# print('GLCM k-NN Accuracy: {:.4f}'.format(accuracy_score(y_glcm_test, y_glcm_pred)))

# y_glcm_pred = svm(X_glcm_train, y_glcm_train, X_glcm_test, props=svmProps)
# print('GLCM SVM Accuracy: {:.4f}'.format(accuracy_score(y_glcm_test, y_glcm_pred)))


# # ========================= LBP
# # X_lbp_train, X_lbp_test, y_lbp_train, y_lbp_test = split_normalizado(X_lbp, y, test_size=0.3)
# X_lbp_train, X_lbp_test, y_lbp_train, y_lbp_test = split_manual(X_lbp, y, train_size=18)

# y_lbp_pred = knn(X_lbp_train, y_lbp_train, X_lbp_test, props=knnProps)
# print('LBP k-NN Accuracy: {:.4f}'.format(accuracy_score(y_lbp_test, y_lbp_pred)))

# y_lbp_pred = svm(X_lbp_train, y_lbp_train, X_lbp_test, props=svmProps)
# print('LBP SVM Accuracy: {:.4f}'.format(accuracy_score(y_lbp_test, y_lbp_pred)))

In [6]:
for q in [1, 2, 4]:
  #build database
  database = get_files('../datasets/Base_BFL_256/')
  X_glcm, X_lbp, y = createXy(database, quadrants=(q, q))

  #train test split
  X_glcm_train, X_glcm_test, y_glcm_train, y_glcm_test = split_manual(X_glcm, y, train_size=18)
  X_lbp_train, X_lbp_test, y_lbp_train, y_lbp_test = split_manual(X_lbp, y, train_size=18)
  
  print(f'-------------------- quadrantes {q}x{q}')

  #knn
  for metric in ['cityblock', 'euclidean']:
    for k in [1, 3, 5]:
  
      knnProps={
        'n_neighbors': k,
        'metric': metric,
      }

      print(f'============= KNN TEST: quadr={q}x{q}, distance={metric}, k={k}')

      y_glcm_pred = knn(X_glcm_train, y_glcm_train, X_glcm_test, props=knnProps)
      print('GLCM k-NN Accuracy: {:.4f}'.format(accuracy_score(y_glcm_test, y_glcm_pred)))
    
      y_lbp_pred = knn(X_lbp_train, y_lbp_train, X_lbp_test, props=knnProps)
      print('LBP k-NN Accuracy: {:.4f}'.format(accuracy_score(y_lbp_test, y_lbp_pred)))
  #end knn

  #svm
  for kernel in ['rbf', 'linear']:
    for c in (2. ** np.arange(-5,15,6)):
      for gamma in (2. ** np.arange(3,-15,-6)):
        
        svmProps={
          'C': c,
          'kernel': kernel,
          'gamma': gamma,
          'coef0': 0
        }
      
        print(f'============= SVM TEST: quadr={q}x{q}, kernel={kernel}, C={c}, gamma={gamma}')

        y_glcm_pred = svm(X_glcm_train, y_glcm_train, X_glcm_test, props=svmProps)
        print('GLCM SVM Accuracy: {:.4f}'.format(accuracy_score(y_glcm_test, y_glcm_pred)))

        y_lbp_pred = svm(X_lbp_train, y_lbp_train, X_lbp_test, props=svmProps)
        print('LBP SVM Accuracy: {:.4f}'.format(accuracy_score(y_lbp_test, y_lbp_pred)))
  #end svm

#end q

getfiles done
54 531


54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 530
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 530
54 531
54 531
54 530
54 531
54 531
54 531
54 531
54 531
54 531
54 529
54 529
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531
54 531

KeyboardInterrupt: 